<a href="https://colab.research.google.com/github/Hbrand03/Real-Time-Machine-Learning/blob/main/Homework_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Problem 1

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the improved AlexNet
class ImprovedAlexNet(nn.Module):
    def __init__(self, num_classes=10, dropout_rate=0.6):
        super(ImprovedAlexNet, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),  # Batch Normalization
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),  # Increased Dropout

            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Apply data augmentation for better generalization
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(trainset, batch_size=128, shuffle=True)
test_loader = DataLoader(testset, batch_size=128, shuffle=False)

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Model
model = ImprovedAlexNet(num_classes=10, dropout_rate=0.6).to(device)

# Define Loss and Optimizer with L2 Regularization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Function
def train(model, train_loader, criterion, optimizer, scheduler, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()  # Adjust learning rate

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Train the model
train(model, train_loader, criterion, optimizer, scheduler)

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Loss: {loss/len(test_loader):.4f}, Accuracy: {accuracy:.2f}%")

# Test the model
evaluate(model, test_loader)


100%|██████████| 170M/170M [00:26<00:00, 6.52MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/20], Loss: 1.6298
Epoch [2/20], Loss: 1.2739
Epoch [3/20], Loss: 1.1023
Epoch [4/20], Loss: 0.9767
Epoch [5/20], Loss: 0.8850
Epoch [6/20], Loss: 0.7423
Epoch [7/20], Loss: 0.6789
Epoch [8/20], Loss: 0.6423
Epoch [9/20], Loss: 0.6100
Epoch [10/20], Loss: 0.5763
Epoch [11/20], Loss: 0.4970
Epoch [12/20], Loss: 0.4645
Epoch [13/20], Loss: 0.4383
Epoch [14/20], Loss: 0.4204
Epoch [15/20], Loss: 0.4045
Epoch [16/20], Loss: 0.3580
Epoch [17/20], Loss: 0.3370
Epoch [18/20], Loss: 0.3235
Epoch [19/20], Loss: 0.3148
Epoch [20/20], Loss: 0.3039
Validation Loss: 0.4333, Accuracy: 86.02%


# Problem 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

# Define the modified VGG-11 model
class VGG11_Custom(nn.Module):
    def __init__(self, num_classes=10, dropout_rate=0.6):
        super(VGG11_Custom, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),

            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * 4 * 4, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),

            nn.Linear(512, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Apply data augmentation
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load CIFAR-10 dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(trainset, batch_size=128, shuffle=True)
test_loader = DataLoader(testset, batch_size=128, shuffle=False)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize Model
model = VGG11_Custom(num_classes=10, dropout_rate=0.6).to(device)

# Define Loss and Optimizer with L2 Regularization
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

# Learning rate scheduler
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.5)

# Training Function
def train(model, train_loader, criterion, optimizer, scheduler, num_epochs=20):
    model.train()
    for epoch in range(num_epochs):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        scheduler.step()  # Adjust learning rate

        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

# Train the model
train(model, train_loader, criterion, optimizer, scheduler)

# Evaluate the model
def evaluate(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    loss = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss += criterion(outputs, labels).item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f"Validation Loss: {loss/len(test_loader):.4f}, Accuracy: {accuracy:.2f}%")

# Test the model
evaluate(model, test_loader)


100%|██████████| 170M/170M [00:42<00:00, 4.00MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Epoch [1/20], Loss: 1.6216
Epoch [2/20], Loss: 1.2650
Epoch [3/20], Loss: 1.0960
Epoch [4/20], Loss: 0.9813
Epoch [5/20], Loss: 0.8892
Epoch [6/20], Loss: 0.7354
Epoch [7/20], Loss: 0.6857
Epoch [8/20], Loss: 0.6512
Epoch [9/20], Loss: 0.6081
Epoch [10/20], Loss: 0.5788
Epoch [11/20], Loss: 0.4986
Epoch [12/20], Loss: 0.4696
Epoch [13/20], Loss: 0.4543
Epoch [14/20], Loss: 0.4273
Epoch [15/20], Loss: 0.4121
Epoch [16/20], Loss: 0.3656
Epoch [17/20], Loss: 0.3485
Epoch [18/20], Loss: 0.3306
Epoch [19/20], Loss: 0.3255
Epoch [20/20], Loss: 0.3116
Validation Loss: 0.4333, Accuracy: 85.95%


# Problem 3

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

# ------------------------
# DATA LOADING & AUGMENTATION
# ------------------------
transform_train = transforms.Compose([
    transforms.ToTensor(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomErasing(p=0.5, scale=(0.02, 0.2)),
    transforms.Normalize((0.5,), (0.5,))
])


transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

batch_size = 128

trainset_c10 = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
train_loader_c10 = DataLoader(trainset_c10, batch_size=batch_size, shuffle=True)

testset_c10 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
test_loader_c10 = DataLoader(testset_c10, batch_size=batch_size, shuffle=False)

trainset_c100 = torchvision.datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
train_loader_c100 = DataLoader(trainset_c100, batch_size=batch_size, shuffle=True)

testset_c100 = torchvision.datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)
test_loader_c100 = DataLoader(testset_c100, batch_size=batch_size, shuffle=False)

# ------------------------
# DEFINE RESNET-18 MODEL
# ------------------------
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        return F.relu(out)

class ResNet18(nn.Module):
    def __init__(self, num_classes=10, dropout=0.4):
        super(ResNet18, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(64, 2, stride=1)
        self.layer2 = self._make_layer(128, 2, stride=2)
        self.layer3 = self._make_layer(256, 2, stride=2)
        self.layer4 = self._make_layer(512, 2, stride=2)
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)
        self.dropout = nn.Dropout(dropout)

    def _make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(BasicBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(BasicBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.avgpool(out)
        out = torch.flatten(out, 1)
        out = self.dropout(out)
        return self.fc(out)

# ------------------------
# TRAINING FUNCTION
# ------------------------
def train(model, train_loader, test_loader, epochs=20, lr=0.0005, weight_decay=5e-4, num_classes=10):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)

    criterion = nn.CrossEntropyLoss()
    train_losses, val_losses, val_accs = [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        correct, total = 0, 0
        val_loss = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = val_loss / len(test_loader)
        val_losses.append(avg_val_loss)
        accuracy = 100 * correct / total
        val_accs.append(accuracy)

        print(f"Epoch [{epoch+1}/{epochs}], Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Accuracy: {accuracy:.2f}%")
        scheduler.step()

    return train_losses, val_losses, val_accs

# ------------------------
# TRAIN MODELS
# ------------------------
# Train CIFAR-10
model_c10 = ResNet18(num_classes=10, dropout=0.4)
train_losses_c10, val_losses_c10, val_accs_c10 = train(model_c10, train_loader_c10, test_loader_c10, epochs=20, lr=0.0005)

# Train CIFAR-100
model_c100 = ResNet18(num_classes=100, dropout=0.4)
train_losses_c100, val_losses_c100, val_accs_c100 = train(model_c100, train_loader_c100, test_loader_c100, epochs=20, lr=0.0005)

# ------------------------
# PLOT RESULTS
# ------------------------
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.plot(val_accs_c10, label="CIFAR-10")
plt.plot(val_accs_c100, label="CIFAR-100")
plt.xlabel("Epochs")
plt.ylabel("Validation Accuracy")
plt.title("Validation Accuracy Over Epochs")
plt.legend()

plt.subplot(1,2,2)
plt.plot(val_losses_c10, label="CIFAR-10")
plt.plot(val_losses_c100, label="CIFAR-100")
plt.xlabel("Epochs")
plt.ylabel("Validation Loss")
plt.title("Validation Loss Over Epochs")
plt.legend()

plt.show()


Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


KeyboardInterrupt: 